In [1]:
from er_evaluation.datasets import load_pv_disambiguations, load_pv_data

predictions, reference = load_pv_disambiguations()

pv_data = load_pv_data()
pv_data.set_index("mention_id", inplace=True)
names = pv_data["raw_inventor_name_first"] + " " + pv_data["raw_inventor_name_last"]

## Summary Statistics

In [2]:
from er_evaluation.plots import plot_summaries

plot_summaries(predictions, names)

## Performance Estimates

In [3]:
from er_evaluation.plots import plot_estimates
from er_evaluation.estimators import *

fig = plot_estimates(predictions, {"sample": reference, "weights": "cluster_size"})

fig

## Disambiguation Similarity

In [4]:
from er_evaluation.plots import plot_comparison

plot_comparison(predictions)

## Cluster Error Metrics

In [5]:
from er_evaluation.plots import plot_cluster_errors

weights = 1/reference.value_counts()

plot_cluster_errors(predictions[list(predictions.keys())[0]], reference, weights=weights)

## Fairness Analysis

In [6]:
import pandas as pd
import numpy as np
from er_evaluation.plots import plot_performance_disparities
from er_evaluation.datasets import load_pv_disambiguations, load_pv_data

predictions, reference = load_pv_disambiguations()
pv_data = load_pv_data()

protected_feature = pv_data.set_index("mention_id")['cpc_section'].apply(lambda x: x[0] if isinstance(x, np.ndarray) and len(x) > 0 else 'None')
protected_feature = pd.concat([reference, protected_feature], join="inner", axis=1).groupby("unique_id").agg("first")["cpc_section"]

plot_performance_disparities(
    prediction=predictions[pd.Timestamp('2021-12-30')],
    reference=reference,
    weights="cluster_size",
    protected_feature=protected_feature,
)